# Saving the point cloud from `Data` to `.xyz` format for VCCS benchmarking

In [8]:
import torch
import os.path as osp
import subprocess


def save_xyz_rgb(xyz: torch.Tensor, rgb: torch.Tensor, path: str):
    """
    Save point cloud to .xyz format (x y z r g b) for C++ supervoxel code.
    
    Args:
        xyz (torch.Tensor): [N, 3], dtype=torch.float32 or float64
        rgb (torch.Tensor): [N, 3], dtype=torch.uint8 (0-255)
        path (str): Output file path.
    """
    assert xyz.ndim == 2 and xyz.shape[1] == 3, "xyz must be [N,3]"
    assert rgb.ndim == 2 and rgb.shape[1] == 3, "rgb must be [N,3]"
    assert xyz.shape[0] == rgb.shape[0], "xyz and rgb must have same N"
    assert rgb.dtype == torch.uint8, "rgb must be uint8"

    # Convert to CPU numpy arrays
    xyz_np = xyz.cpu().numpy()
    rgb_np = rgb.cpu().numpy()

    with open(path, 'w') as f:
        for p, c in zip(xyz_np, rgb_np):
            f.write(f"{p[0]} {p[1]} {p[2]} {int(c[0])} {int(c[1])} {int(c[2])}\n")

    print(f"Saved {xyz.shape[0]} points to {path}")


def read_xyz_rgb_label(path: str):
    """
    Reads an XYZ+RGB+label file into torch tensors.

    Args:
        path (str): Path to the .xyz file.

    Returns:
        xyz: Float tensor of shape [N, 3]
        rgb: UInt8 tensor of shape [N, 3]
        labels: Long tensor of shape [N]
    """
    # Load everything as float first
    data = torch.tensor([list(map(float, line.strip().split()))
                         for line in open(path) if line.strip()], dtype=torch.float)

    if data.shape[1] != 7:
        raise ValueError(f"Expected 7 columns in the file (x y z r g b label), got {data.shape[1]}")

    xyz = data[:, :3].clone()           # x, y, z
    rgb = data[:, 3:6].to(torch.uint8)  # r, g, b
    labels = data[:, 6].to(torch.long)  # label index

    return xyz, rgb, labels

In [9]:
# Example command with arguments
resolution = 1.0
root_dir = "/home/damien/projects/supervoxel-for-3d-point-clouds"
save_vccs_output_to_disk = True

input_file = osp.join(root_dir, 'test_data', 'xyzrgb_area5_voxelized_3cm.xyz')
output_file = osp.join(root_dir, 'test_data', 'xyzrgb_area5_voxelized_3cm_out_vccs.xyz')

In [10]:
# To save an existing Data object to the input file format for VCCS
# save_xyz_rgb(data.pos, data.rgb, input_file)

In [11]:
# Construct the command as a list of strings
cmd = [osp.join(root_dir, "superpoints"), input_file, str(resolution)] + ['--save'] * save_vccs_output_to_disk

# Run the command
result = subprocess.run(cmd, capture_output=True, text=True)

# Print stdout and stderr
print("Output:\n", result.stdout)
print("Errors:\n", result.stderr)

# Check return code
if result.returncode != 0:
    print("Command failed")

Output:
 I0815 12:03:58.424 main.cc:87] Using file: /home/damien/projects/supervoxel-for-3d-point-clouds/test_data/xyzrgb_area5_voxelized_3cm.xyz
I0815 12:03:58.424 main.cc:88] Using resolution: 1
I0815 12:03:58.424 main.cc:99] Reading points from /home/damien/projects/supervoxel-for-3d-point-clouds/test_data/xyzrgb_area5_voxelized_3cm.xyz...
I0815 12:04:06.015 main.cc:106] 9307791 points are imported.
I0815 12:04:06.015 main.cc:107] Time for reading points: 7591 ms
I0815 12:04:06.015 main.cc:113] Building KD tree...
I0815 12:04:17.979 main.cc:132] Time for KD-tree + KNN + normals: 11963 ms
I0815 12:04:17.979 main.cc:138] Start SupervoxelSegmentation...
I0815 12:04:28.678 main.cc:155] Time for SupervoxelSegmentation: 10699 ms
I0815 12:04:28.678 main.cc:158] Input points: 9307791, Output superpoints: 6862
I0815 12:04:28.678 main.cc:163] Start VCCS supervoxel segmentation...
I0815 12:04:47.333 main.cc:172] Time for VCCS: 18654 ms
I0815 12:04:47.333 main.cc:175] Input points: 9307791, Out

In [12]:
xyz, rgb_for_sp_viz, super_index = read_xyz_rgb_label(output_file)